In [1]:
from models import deck
from scipy.stats import multivariate_hypergeom, hypergeom


Specify the filesystem path in `filepath` variable, then load the deck into memory

In [2]:
filepath = "./deck_example.txt"
mydeck = deck.Deck.from_file(filepath)

./deck_example.txt


Validate the total cards of each type:

In [3]:
def total_count(cards):
    return sum([
        i.count
        for i in cards
    ])

bricks = mydeck.cards_by_deck_type(deck.DeckCardType.BRICK)
non_engine = mydeck.cards_by_deck_type(deck.DeckCardType.NONENGINE)
engine = mydeck.cards_by_deck_type(deck.DeckCardType.ENGINE)

print(f"Your deck has {total_count(bricks)} bricks, {total_count(non_engine)} non-engine, and {total_count(engine)} engine out of {mydeck.size} cards")

Your deck has 6 bricks, 14 non-engine, and 20 engine out of 40 cards


Now find out the chances you see any particular combo:

In [4]:
for combo in mydeck.combos:
    m_arr = [ card.count for card in combo.cards ]
    m_arr.append(mydeck.size - len(combo.cards))
    x_arr = [ 1 for card in combo.cards]
    x_arr.append(5 - len(combo.cards))
    rv = multivariate_hypergeom(m=m_arr, n=5)
    print(f"{combo}: {rv.pmf(x=x_arr) * 100}% chance to see")

madolche_anjelly: 29.006968641114945% chance to see
madolche_petingcessour: 29.006968641114945% chance to see
madolche_anjelly + madolche_petingcessour: 6.9911087211143945% chance to see
madolche_hootcake + vernusylph_of_the_misting_seedlings: 5.258269807333905% chance to see
madolche_hootcake + vernusylph_of_the_flourishing_hills: 3.966764942374694% chance to see
madolche_petingcessour + vernusylph_of_the_misting_seedlings: 6.9911087211143945% chance to see
madolche_petingcessour + emergency_teleport: 6.9911087211143945% chance to see


Chance to see any 1 card combo:

In [24]:
one_card = list(filter(lambda x: len(x.cards) == 1, mydeck.combos))
one_card_cards = [ card for combo in one_card for card in combo.cards]
total_one_card = sum([
    i.count
    for combo in one_card for i in combo.cards
])
rv = multivariate_hypergeom(m=[total_one_card, mydeck.size - total_one_card], n=5)
print(f"All 1 card combos: {one_card_cards}")
print(f"You'll see a 1 card combo in {rv.pmf(x=[1,4])*100}% of hands")


All 1 card combos: [madolche_anjelly, madolche_petingcessour]
You'll see a 1 card combo in 42.287631761315986% of hands


Chance to see any 2 card combo:
1. Find all discrete cards in all the combos
2. Calculate the probability that the hand has 0 of the cards in the combo
3. For each combo, calculate that only one of the cards is present in the hand

In [25]:
two_card = list(filter(lambda x: len(x.cards) == 2, mydeck.combos))
cards_in_two_card = set(
    [
        card for combo in two_card for card in combo.cards
    ]
)
# Find hands where none of the cards from the combo are in it at all
total_no_card = sum([
    i.count for i in cards_in_two_card
])
no_cards = hypergeom.cdf(0, mydeck.size, total_no_card, 5)
# For each combo, calculate the odds of seeing one and not the other
for combo in two_card:
    print("--------------")
    print(f"Combo: {combo}")
    # Chance to see one but not both cards
    # Take 1 of either card, and then 4 of not the other
    total_card = combo.cards[0].count
    total_other_card = combo.cards[1].count
    rv_1 = multivariate_hypergeom(m=[total_card, mydeck.size - total_other_card], n=5)
    p_1 = rv_1.pmf(x=[1,4])
    rv_2 = multivariate_hypergeom(m=[total_other_card, mydeck.size - total_card], n=5)
    p_2 = rv_1.pmf(x=[1,4])
    chance_to_miss = p_1 + p_2
    print(f"Chance to miss: {chance_to_miss*100}%")


--------------
Combo: madolche_anjelly + madolche_petingcessour
Chance to miss: 60.222672064777264%
--------------
Combo: madolche_hootcake + vernusylph_of_the_misting_seedlings
Chance to miss: 45.88394062078272%
--------------
Combo: madolche_hootcake + vernusylph_of_the_flourishing_hills
Chance to miss: 44.871794871794805%
--------------
Combo: madolche_petingcessour + vernusylph_of_the_misting_seedlings
Chance to miss: 60.222672064777264%
--------------
Combo: madolche_petingcessour + emergency_teleport
Chance to miss: 60.222672064777264%
